In [93]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from IPython.display import Image, display_png
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

In [94]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [95]:
v = t.T[[
    'R01786','R02740','R04779','R01070','R01061','R01512','R01518','R00658',
    'R00200','R01196','R00351','R00341','R01325','R01900','R01899','R00268',
    'R01197','R00405','R02164','R01082','R00342','R01015','R00703'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [96]:
df = v.T
df['Glucose'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose']['C00267'] = 1

df['Oxygen'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen']['C00007'] = 1

df['CO2'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2']['C00011'] = -1

df['Lactate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate']['C00186'] = -1

df['Acetate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate']['C00033'] = -1

# acetyl-CoA hydrolase  # acetyl-CoA + H2O = CoA + acetate
df['acetyl-CoA hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl-CoA hydrolase']['C00024'] = -1
df['acetyl-CoA hydrolase']['C00001'] = -1
df['acetyl-CoA hydrolase']['C00010'] = 1
df['acetyl-CoA hydrolase']['C00033'] = 1

# アミノ酸 αKG 核酸 G6P Phopphatydylcholin コレステロール Acoa 比率 

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

# Oxophos
df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

df['Biomass'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Biomass']['C00002'] = -1
df['Biomass']['C00008'] = 1

df = df.drop([
    # H2O 
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603'
])

In [104]:
x = []
for i in df.columns:
    if (
        (i=='Biomass') or (i=='Oxophos') or (i=='R01899') or (i=='R00268') or 
        (i=='R10343') or (i=='R02164') or (i=='CO2') or (i == 'Lactate') or (i == 'Acetate')
    ) :
        x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))
        
    elif (i=='Glucose'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
    elif (i=='Oxygen'):
        x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
        
    elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
        x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))
    
    else:
        x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

In [105]:
m = LpProblem(sense=LpMaximize)
m += x[-1]

for i in np.dot(df, x):
    m += i == 0

In [106]:
# s = df.T[['C00003','C00004','C00005','C00006']].T
# for i in s.columns:
#     if (sum(s[i] == 0.0) == 4):
#         s = s.drop(i,axis=1)
# s

In [107]:
# df.T['C00002'][(df.T['C00002'] != 0.0)]

In [108]:
m

NoName:
MAXIMIZE
1*Biomass + 0
SUBJECT TO
_C1: - Biomass + 5 Oxophos - R00200 - R00341 - R00405 - R01512 - R01786
 - R04779 = 0

_C2: - NADP_Exchange + 2 Oxophos - R00342 - R00703 - R01061 + R02164 = 0

_C3: NADP_Exchange - 2 Oxophos + R00342 + R00703 + R01061 - R02164 = 0

_C4: - NADP_Exchange + R01899 = 0

_C5: NADP_Exchange - R01899 = 0

_C6: Biomass - 5 Oxophos + R00200 + R00341 + R00405 + R01512 + R01786 + R04779
 = 0

_C7: - R00351 - R00405 + R01196 + R01197 + acetyl_CoA_hydrolase = 0

_C8: - CO2 + Oxophos + R00268 + R00341 - R01196 - R01197 = 0

_C9: - R00200 + R00703 + R01196 = 0

_C10: R00351 - R01196 - acetyl_CoA_hydrolase = 0

_C11: R00268 + R01197 = 0

_C12: - R00341 + R00342 + R00351 = 0

_C13: - R00405 - R02164 = 0

_C14: R00200 + R00341 + R00658 = 0

_C15: R00405 - R01197 = 0

_C16: R01015 + R01070 = 0

_C17: - R01015 - R01061 + R01070 = 0

_C18: R01082 + R02164 = 0

_C19: - R00342 - R01082 = 0

_C20: - R00351 - R01325 = 0

_C21: - Lactate - R00703 = 0

_C22: - R01512 + 

In [109]:
status = m.solve()
print(LpStatus[status])

Optimal


In [110]:
y = []
for i in x:
    y.append(i.value())
    
pd.DataFrame(y, index=x)

,0
R01786,100.0
R02740,100.0
R04779,100.0
R01070,100.0
R01061,200.0
R01512,-200.0
R01518,-200.0
R00658,200.0
R00200,-200.0
R01196,-200.0
